# Example of the SASRec training and inference stages
Note that all the given examples can be run without using PySpark, using only Pandas

In [1]:
import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
import torch

from replay.metrics import OfflineMetrics, Recall, Precision, MAP, NDCG, HitRate, MRR
from replay.metrics.torch_metrics_builder import metrics_to_df
from replay.splitters import LastNSplitter
from replay.data import (
    FeatureHint,
    FeatureInfo,
    FeatureSchema,
    FeatureSource,
    FeatureType,
    Dataset,
)
from replay.models.nn.optimizer_utils import FatOptimizerFactory
from replay.models.nn.sequential.callbacks import (
    ValidationMetricsCallback,
    SparkPredictionCallback,
    PandasPredictionCallback, 
    TorchPredictionCallback,
    QueryEmbeddingsPredictionCallback
)
from replay.models.nn.sequential.postprocessors import RemoveSeenItems
from replay.data.nn import (
    SequenceTokenizer,
    SequentialDataset,
    TensorFeatureSource,
    TensorSchema,
    TensorFeatureInfo
)
from replay.models.nn.sequential import SasRec
from replay.models.nn.sequential.sasrec import (
    SasRecPredictionDataset,
    SasRecTrainingDataset,
    SasRecValidationDataset,
    SasRecPredictionBatch,
    SasRecModel
)

import pandas as pd
import polars as pl

## Prepare data
### Load raw movielens-1M interactions, item features and user features.
In the current implementation, the SASRec does not take into account the features of items or users. They are only used to get a complete list of users and items.

In [2]:
!pip install rs-datasets

In [3]:
from rs_datasets import MovieLens

In [4]:
movielens = MovieLens("1m")
interactions = movielens.ratings
user_features = movielens.users
item_features = movielens.items

In [5]:
import numpy as np
from replay.preprocessing.filters import NumInteractionsFilter

In [6]:
# interactions = NumInteractionsFilter(num_interactions=5).transform(interactions)
sz = interactions.shape[0]
cols = 8
interactions["int_cat_list"] = np.random.randint(0, 10, size=(sz, cols)).tolist()
interactions["int_num_list"] = np.random.randn(sz, cols).tolist()
interactions["int_cat"] = np.random.randint(0, 10, size=sz)
interactions["int_num"] = np.random.randn(sz)
# interactions = pl.DataFrame(interactions)

In [7]:
interactions

,user_id,item_id,rating,timestamp,int_cat_list,int_num_list,int_cat,int_num
0,1,1193,5,978300760,"[4, 7, 5, 3, 9, 6, 6, 5]","[-0.27816860643294294, -0.9402160242022638, -0...",8,-0.943582
1,1,661,3,978302109,"[8, 7, 1, 3, 5, 9, 9, 9]","[0.32672808021581884, 0.14133843739609267, -1....",7,-0.779098
2,1,914,3,978301968,"[5, 5, 9, 7, 9, 0, 3, 8]","[0.46296375108840315, -0.2595300691964234, 0.1...",9,0.601657
3,1,3408,4,978300275,"[7, 7, 4, 7, 4, 0, 1, 4]","[1.8709501277109173, -0.6457708708502996, 1.31...",6,-2.331955
4,1,2355,5,978824291,"[1, 4, 3, 3, 6, 9, 1, 3]","[0.3650871372795643, 0.04014247551687752, 0.34...",6,0.965065
...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,"[7, 2, 6, 5, 7, 2, 8, 3]","[1.4377177589242183, -0.10815416627056858, -0....",6,-0.758533
1000205,6040,1094,5,956704887,"[9, 8, 5, 6, 7, 0, 9, 2]","[1.2794104039537237, 0.9896057400696546, 1.897...",1,-1.486710
1000206,6040,562,5,956704746,"[1, 6, 9, 3, 6, 5, 6, 5]","[-0.01008738778070476, 1.2067734905656247, 0.5...",0,0.707191
1000207,6040,1096,4,956715648,"[6, 3, 6, 2, 1, 2, 6, 1]","[-0.8476870361541977, 1.491881104413442, 0.307...",6,-0.604098


In [8]:
from replay.preprocessing import GroupedLabelEncodingRule, LabelEncoder

le = LabelEncoder([GroupedLabelEncodingRule("int_cat_list")])
le.fit(interactions)
le.mapping

{'int_cat_list': {4: 0, 7: 1, 5: 2, 3: 3, 9: 4, 6: 5, 8: 6, 1: 7, 0: 8, 2: 9}}

In [10]:
res = le.transform(interactions)
res

In [ ]:
sz = user_features.shape[0]
cols = 6
user_features["user_cat_list"] = np.random.randint(0, 10, size=(sz, cols)).tolist()
user_features["user_num_list"] = np.random.randn(sz, cols).tolist()
user_features["user_cat"] = np.random.randint(0, 10, size=sz)
user_features["user_num"] = np.random.randn(sz)
# user_features = pl.DataFrame(user_features)
user_features

In [ ]:
sz = item_features.shape[0]
cols = 13
item_features.sort_values("item_id", inplace=True)
item_features["item_cat_list"] = np.random.randint(0, 10, size=(sz, cols)).tolist()
item_features["item_num_list"] = np.random.randn(sz, cols).tolist()
item_features["item_cat"] = np.random.randint(0, 10, size=sz).tolist()
item_features["item_num"] = np.random.randn(sz).tolist()
# item_features = pl.DataFrame(item_features)

In [ ]:
item_features

Removing duplicates in the timestamp column without changing the original items order where timestamp is the same

In [ ]:
# interactions["timestamp"] = interactions["timestamp"].astype("int64")
# interactions = interactions.sort_values(by="timestamp")
# interactions["timestamp"] = interactions.groupby("user_id").cumcount()

### Split interactions into the train, validation and test datasets using LastNSplitter

In [ ]:
splitter = LastNSplitter(
    N=1,
    divide_column="user_id",
    query_column="user_id",
    strategy="interactions",
)

raw_test_events, raw_test_gt = splitter.split(interactions)
raw_validation_events, raw_validation_gt = splitter.split(raw_test_events)
raw_train_events = raw_validation_events

### Prepare FeatureSchema required to create Dataset

In [ ]:
def prepare_feature_schema(is_ground_truth: bool) -> FeatureSchema:
    all_features = FeatureSchema(
        [
            FeatureInfo(
                column="user_id",
                feature_hint=FeatureHint.QUERY_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
            FeatureInfo(
                column="item_id",
                feature_hint=FeatureHint.ITEM_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
            FeatureInfo(
                column="int_cat_list",
                feature_type=FeatureType.CATEGORICAL_LIST,
            ),
            FeatureInfo(
                column="int_num_list",
                feature_type=FeatureType.NUMERICAL_LIST,
            ),
            FeatureInfo(
                column="int_cat",
                feature_type=FeatureType.CATEGORICAL,
            ),
            FeatureInfo(
                column="int_num",
                feature_type=FeatureType.NUMERICAL,
            ),
        ]
    )
    if is_ground_truth:
        return all_features

    all_features = all_features + FeatureSchema(
        [
            FeatureInfo(
                column="timestamp",
                feature_type=FeatureType.NUMERICAL,
                feature_hint=FeatureHint.TIMESTAMP,
            ),
            # item features
            FeatureInfo(
                column="item_cat_list",
                feature_type=FeatureType.CATEGORICAL_LIST,
                feature_source=FeatureSource.ITEM_FEATURES,
            ),
            FeatureInfo(
                column="item_num_list",
                feature_type=FeatureType.NUMERICAL_LIST,
                feature_source=FeatureSource.ITEM_FEATURES,
            ),
            FeatureInfo(
                column="item_cat",
                feature_type=FeatureType.CATEGORICAL,
                feature_source=FeatureSource.ITEM_FEATURES,
            ),
            FeatureInfo(
                column="item_num",
                feature_type=FeatureType.NUMERICAL,
                feature_source=FeatureSource.ITEM_FEATURES,
            ),
            # query features
            FeatureInfo(
                column="user_cat_list",
                feature_type=FeatureType.CATEGORICAL_LIST,
                feature_source=FeatureSource.QUERY_FEATURES,
            ),
            FeatureInfo(
                column="user_num_list",
                feature_type=FeatureType.NUMERICAL_LIST,
                feature_source=FeatureSource.QUERY_FEATURES,
            ),
            FeatureInfo(
                column="user_cat",
                feature_type=FeatureType.CATEGORICAL,
                feature_source=FeatureSource.QUERY_FEATURES,
            ),
            FeatureInfo(
                column="user_num",
                feature_type=FeatureType.NUMERICAL,
                feature_source=FeatureSource.QUERY_FEATURES,
            ),
        ]
    )
    return all_features

### Create Dataset for the training stage

In [ ]:
train_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_train_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)

### Create Datasets (events and ground_truth) for the validation stage

In [ ]:
validation_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_validation_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
validation_gt = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=True),
    interactions=raw_validation_gt,
    check_consistency=True,
    categorical_encoded=False,
)

### Create Datasets (events and ground_truth) for the testing stage

In [ ]:
test_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_test_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
test_gt = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=True),
    interactions=raw_test_gt,
    check_consistency=True,
    categorical_encoded=False,
)

### Create the tensor schema
A schema shows the correspondence of columns from the source dataset with the internal representation of tensors inside the model

In [ ]:
ITEM_FEATURE_NAME = "item_id_seq"

tensor_schema = TensorSchema([
    TensorFeatureInfo(
        name=ITEM_FEATURE_NAME,
        is_seq=True,
        feature_type=FeatureType.CATEGORICAL,
        feature_source=TensorFeatureSource(FeatureSource.INTERACTIONS, train_dataset.feature_schema.item_id_column),
        feature_hint=FeatureHint.ITEM_ID,
    ),
    TensorFeatureInfo(
        name="timestamp_seq",
        is_seq=True,
        feature_type=FeatureType.NUMERICAL,
        feature_source=TensorFeatureSource(FeatureSource.INTERACTIONS, "timestamp"),
        feature_hint=FeatureHint.TIMESTAMP,
    ),
    TensorFeatureInfo(
        name="item_cat_list_seq",
        is_seq=True,
        feature_type=FeatureType.CATEGORICAL_LIST,
        feature_source=TensorFeatureSource(FeatureSource.ITEM_FEATURES, "item_cat_list"),
    ),
    TensorFeatureInfo(
        name="item_num_list_seq",
        is_seq=True,
        feature_type=FeatureType.NUMERICAL_LIST,
        feature_source=TensorFeatureSource(FeatureSource.ITEM_FEATURES, "item_num_list"),
    ),
    TensorFeatureInfo(
        name="item_cat_seq",
        is_seq=True,
        feature_type=FeatureType.CATEGORICAL,
        feature_source=TensorFeatureSource(FeatureSource.ITEM_FEATURES, "item_cat"),
    ),
    TensorFeatureInfo(
        name="item_num_seq",
        is_seq=True,
        feature_type=FeatureType.NUMERICAL,
        feature_source=TensorFeatureSource(FeatureSource.ITEM_FEATURES, "item_num"),
    ),
    TensorFeatureInfo(
        name="int_cat_list_seq",
        is_seq=True,
        feature_type=FeatureType.CATEGORICAL_LIST,
        feature_source=TensorFeatureSource(FeatureSource.INTERACTIONS, "int_cat_list"),
    ),
    TensorFeatureInfo(
        name="int_num_list_seq",
        is_seq=True,
        feature_type=FeatureType.NUMERICAL_LIST,
        feature_source=TensorFeatureSource(FeatureSource.INTERACTIONS, "int_num_list"),
    ),
    TensorFeatureInfo(
        name="int_cat_seq",
        is_seq=True,
        feature_type=FeatureType.CATEGORICAL,
        feature_source=TensorFeatureSource(FeatureSource.INTERACTIONS, "int_cat"),
    ),
    TensorFeatureInfo(
        name="int_num_seq",
        is_seq=True,
        feature_type=FeatureType.NUMERICAL,
        feature_source=TensorFeatureSource(FeatureSource.INTERACTIONS, "int_num"),
    ),
    TensorFeatureInfo(
        name="user_cat_list_seq",
        is_seq=True,
        feature_type=FeatureType.CATEGORICAL_LIST,
        feature_source=TensorFeatureSource(FeatureSource.QUERY_FEATURES, "user_cat_list"),
    ),
    TensorFeatureInfo(
        name="user_cat_list_not_seq",
        is_seq=False,
        feature_type=FeatureType.CATEGORICAL_LIST,
        feature_source=TensorFeatureSource(FeatureSource.QUERY_FEATURES, "user_cat_list"),
    ),
    TensorFeatureInfo(
        name="user_cat_seq",
        is_seq=True,
        feature_type=FeatureType.CATEGORICAL,
        feature_source=TensorFeatureSource(FeatureSource.QUERY_FEATURES, "user_cat"),
    ),
    TensorFeatureInfo(
        name="user_cat_not_seq",
        is_seq=False,
        feature_type=FeatureType.CATEGORICAL,
        feature_source=TensorFeatureSource(FeatureSource.QUERY_FEATURES, "user_cat"),
    ),
])

### Create sequential datasets using SequenceTokenizer
The SequentialDataset internally store data in the form of sequences of items sorted by increasing interaction time (timestamp). A SequenceTokenizer is used to convert to this format. In addition, the SequenceTokenizer encodes all categorical columns from the source dataset and stores mapping inside itself.
SequentialDataset.keep_common_query_ids is used to leave only sequences from the same users

In [ ]:
tokenizer = SequenceTokenizer(tensor_schema, allow_collect_to_master=True)
tokenizer.fit(train_dataset)

sequential_train_dataset = tokenizer.transform(train_dataset)

sequential_validation_dataset = tokenizer.transform(validation_dataset)
sequential_validation_gt = tokenizer.transform(validation_gt, [tensor_schema.item_id_feature_name])

sequential_validation_dataset, sequential_validation_gt = SequentialDataset.keep_common_query_ids(
    sequential_validation_dataset, sequential_validation_gt
)

In [ ]:
sequential_train_dataset._sequences

In [ ]:
seq = sequential_train_dataset.get_sequence(0, "item_id_seq")
print(seq.shape)
seq

In [ ]:
seq = sequential_train_dataset.get_sequence(0, "user_cat_not_seq")
print(seq.shape)
seq

In [ ]:
test_query_ids = test_gt.query_ids
test_query_ids_np = tokenizer.query_id_encoder.transform(test_query_ids)["user_id"].to_numpy()
sequential_test_dataset = tokenizer.transform(test_dataset).filter_by_query_id(test_query_ids_np)

You can get the user and item mapping and inverse mapping as follows

In [ ]:
print(tokenizer.query_id_encoder.mapping, tokenizer.query_id_encoder.inverse_mapping)
print(tokenizer.item_id_encoder.mapping, tokenizer.item_id_encoder.inverse_mapping)

In [ ]:
train_dataloader = DataLoader(
    dataset=SasRecTrainingDataset(
        sequential_train_dataset,
        max_sequence_length=10,
    ),
    batch_size=3,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
)

In [ ]:
for batch in train_dataloader:
    print(batch.features["item_id_seq"])

    print(batch.query_id)
    print(batch.features["user_cat_not_seq"])
    print(batch.features["user_cat_list_not_seq"])
    break

## Train model
### Create SASRec model instance and run the training stage using lightning
After each epoch validation metrics are shown. You can change the list of validation metrics in ValidationMetricsCallback
The model is determined to be the best and is saved if the metric updates its maximum during validation (see the ModelCheckpoint)

In [ ]:
MAX_SEQ_LEN = 200
BATCH_SIZE = 512
NUM_WORKERS = 4

model = SasRec(
    tensor_schema,
    block_count=2,
    head_count=2,
    max_seq_len=MAX_SEQ_LEN,
    hidden_size=300,
    dropout_rate=0.5,
    optimizer_factory=FatOptimizerFactory(learning_rate=0.001),
)

csv_logger = CSVLogger(save_dir=".logs/train", name="SASRec_example")

checkpoint_callback = ModelCheckpoint(
    dirpath=".checkpoints",
    save_top_k=1,
    verbose=True,
    # if you use multiple dataloaders, then add the serial number of the dataloader to the suffix of the metric name.
    # For example,"recall@10/dataloader_idx_0"
    monitor="recall@10",
    mode="max",
)

validation_metrics_callback = ValidationMetricsCallback(
    metrics=["map", "ndcg", "recall"],
    ks=[1, 5, 10, 20],
    item_count=train_dataset.item_count,
    postprocessors=[RemoveSeenItems(sequential_validation_dataset)]
)

trainer = L.Trainer(
    max_epochs=100,
    callbacks=[checkpoint_callback, validation_metrics_callback],
    logger=csv_logger,
)

train_dataloader = DataLoader(
    dataset=SasRecTrainingDataset(
        sequential_train_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
)

validation_dataloader = DataLoader(
    dataset=SasRecValidationDataset(
        sequential_validation_dataset,
        sequential_validation_gt,
        sequential_train_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=validation_dataloader,
)

The path to the best model is saved inside checkpoint_callback

In [ ]:
best_model = SasRec.load_from_checkpoint(checkpoint_callback.best_model_path)

## Inference stage
### Prepare Dataloader and logger

In [ ]:
prediction_dataloader = DataLoader(
    dataset=SasRecPredictionDataset(
        sequential_test_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

csv_logger = CSVLogger(save_dir=".logs/test", name="SASRec_example")

### Run inference
You can get the recommendations in three formats: PySpark DataFrame, Pandas DataFrame, PyTorch tensors. Each of the types corresponds a callback
You can filter the results using postprocessors strategy. For example the RemoveSeenItems postprocessor is filtering out the items that already have been seen in test dataset
You don't need to use all three callbacks. This is shown only for example

Also, you can get user embeddings, that were used to perform predictions, using `get_query_embedding` method inside SasRecModel or `QueryEmbeddingsPredictionCallback` for lightning module.

In [ ]:
TOPK = [1, 10, 20, 100]

postprocessors = [RemoveSeenItems(sequential_test_dataset)]

spark_prediction_callback = SparkPredictionCallback(
    spark_session=spark_session,
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

pandas_prediction_callback = PandasPredictionCallback(
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

torch_prediction_callback = TorchPredictionCallback(
    top_k=max(TOPK),
    postprocessors=postprocessors,
)

query_embeddings_callback = QueryEmbeddingsPredictionCallback()

trainer = L.Trainer(
    callbacks=[
        spark_prediction_callback,
        pandas_prediction_callback,
        torch_prediction_callback,
        query_embeddings_callback
    ], 
    logger=csv_logger, 
    inference_mode=True
)
trainer.predict(best_model, dataloaders=prediction_dataloader, return_predictions=False)

spark_res = spark_prediction_callback.get_result()
pandas_res = pandas_prediction_callback.get_result()
torch_user_ids, torch_item_ids, torch_scores = torch_prediction_callback.get_result()
user_embeddings = query_embeddings_callback.get_result()

In [ ]:
spark_res.show()

In [ ]:
pandas_res

In [ ]:
print(torch_user_ids[0], torch_item_ids[0], torch_scores[0])

Suppose we want to get the recomendations in PySpark format. 
Let's get the inverse representation of labels using inverse_transform method.

Note that the reverse representation can only be obtained for PySpark and Pandas formats. When working with PyTorch tensors, the reverse representation must be done manually

In [ ]:
recommendations = tokenizer.query_and_item_id_encoder.inverse_transform(spark_res)

In [ ]:
recommendations.show()

### Calculating metrics

In [ ]:
init_args = {"query_column": "user_id", "rating_column": "score"}

In [ ]:
result_metrics = OfflineMetrics(
    [Recall(TOPK), Precision(TOPK), MAP(TOPK), NDCG(TOPK), MRR(TOPK), HitRate(TOPK)], **init_args
)(recommendations.toPandas(), raw_test_gt)

In [ ]:
metrics_to_df(result_metrics)

### User embeddings

Got 6040 x 300 user embeddings, because among all 12 batches: 

11 batches contains 512 samples

1 batch contains 408 left samples

11 * 512 + 408 == 6040

In [ ]:
user_embeddings

In [ ]:
user_embeddings.shape

You can access user embeddings directly with `SasRecModel` class

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

core_model = SasRecModel(
    tensor_schema,
    num_blocks=2,
    num_heads=2,
    max_len=MAX_SEQ_LEN,
    hidden_size=300,
    dropout=0.5
)
core_model.eval()
core_model = core_model.to(device)

# Get first batch of data 
data = next(iter(prediction_dataloader))
tensor_map, padding_mask = data.features, data.padding_mask

# Ensure everything is on the same device
padding_mask = padding_mask.to(device)
tensor_map["item_id_seq"] = tensor_map["item_id_seq"].to(device)

# Get user embeddings
user_embeddings_batch = core_model.get_query_embeddings(tensor_map, padding_mask)
user_embeddings_batch

In [ ]:
user_embeddings_batch.shape

### Item embeddings

`get_all_embeddings()` method in transformers can be used to get copies of all embeddings that are presented in model as a dict.

In [ ]:
all_embeddings = best_model.get_all_embeddings()
all_embeddings

You can access item embeddings from this dictionary

In [ ]:
item_embeddings = all_embeddings["item_embedding"]
item_embeddings

Item embeddings shape is (N_ITEMS, HIDDEN_SIZE)

In [ ]:
item_embeddings.shape

Ensure we got correct dimension and ensure we got the copy of tensor

In [ ]:
assert item_embeddings.shape[0] == len(tokenizer.item_id_encoder.mapping["item_id"])
assert id(item_embeddings) != id(best_model._model.item_embedder.item_emb.weight.data)

For example we observe one new item id in our training data. We can easily expand our item embedder by one element

In order to expand item embeddings by new size `set_item_embeddings_by_size` method is applied

In [ ]:
best_model.set_item_embeddings_by_size(item_embeddings.shape[0] + 1)

Now our new item embeddings have one extra embedding

In [ ]:
new_size = best_model.get_all_embeddings()["item_embedding"].shape[0]
old_size = item_embeddings.shape[0]

assert new_size == old_size + 1

Alternatively, we can pass our item embeddings that replace the existing ones by calling `set_item_embeddings_by_tensor`.

If tensor contains new items, they will be added to item embedder.

In [ ]:
new_embeddings_weights = torch.rand((new_size + 1, 300))    # randint used for example only

best_model.set_item_embeddings_by_tensor(new_embeddings_weights)

At the moment we expanded our item embeddings by one more item and replace weights by passing `new_embeddings_weights`

In [ ]:
old_size = new_size
new_size = best_model.get_all_embeddings()["item_embedding"].shape[0]

assert new_size == old_size + 1

Similarly, we can append tensor for only new items with no replace for existing by calling `append_item_embeddings`

In [ ]:
new_item_weights = torch.rand((1, 300))    # randint used for example only

best_model.append_item_embeddings(new_item_weights)

We passed one new example and its weights to item embeddings, thus expanded our vocabulary by one item again

In [ ]:
old_size = new_size
new_size = best_model.get_all_embeddings()["item_embedding"].shape[0]

assert new_size == old_size + 1

## Example of launching an inference for a single user without using a trainer (in order to speed up)
An example for the production of an online script

Let's assume that the user's sequence consisted of a sequence of items [1, 2, 3, 4, 5]. 
Сreate a padding mask corresponding to the sequence of items.

It is important to take only the latest MAX_SEQ_LEN or less items.

In [ ]:
item_sequence = torch.arange(1, 5).unsqueeze(0)[:, -MAX_SEQ_LEN:]
padding_mask = torch.ones_like(item_sequence, dtype=torch.bool)
sequence_item_count = item_sequence.shape[1]

### Wrapping created tensors in the SasRecPredictionBatch entity

In [ ]:
batch = SasRecPredictionBatch(
    query_id=torch.arange(0, item_sequence.shape[0], 1).long(),
    padding_mask=padding_mask.bool(),
    features={ITEM_FEATURE_NAME: item_sequence.long()}
)

### Run predict step of the SasRec and get scores from the model

In [ ]:
with torch.no_grad():
    scores = best_model.predict_step(batch, 0)
scores

### Getting three items with the highest score

In [ ]:
torch.topk(scores, k=3).indices